In [78]:
import pandas as pd
import glob 
import statsmodels.api as sm
import numpy as np

In [22]:
# read all  parquet files in the folder and store them in a dataframe
df = pd.concat([pd.read_parquet(f) for f in glob.glob(r'C:\Users\nasta\OneDrive - TU Eindhoven\Desktop\DCIA\Data-20230329T105410Z-001\Data\*.parquet')], ignore_index = True)
df['date_code'] = pd.to_datetime(df['date_code'], format='%Y%m%d',errors='coerce')


In [3]:
#split df into two dataframes, one for period_code = 3 and 4 and one for period_code = 1 and 2
df_1 = df[df['period_code']==1]
df_2 = df[df['period_code']==2]
df_3 = df[df['period_code']==3]
df_4 = df[df['period_code']==4]

#indicate whater the point is in LP period or not 
df_2['LP'] = 1
df_4['LP'] = 1
df_1['LP'] = 0
df_3['LP'] = 0



C:\Users\nasta\AppData\Local\Temp\ipykernel_32344\2342225733.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['LP'] = 1
C:\Users\nasta\AppData\Local\Temp\ipykernel_32344\2342225733.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_4['LP'] = 1
C:\Users\nasta\AppData\Local\Temp\ipykernel_32344\2342225733.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [150]:
existing_customers = pd.concat([df_1['customer_code'], df_2['customer_code'], df_3['customer_code']])
df_4['new_customer'] = ~df_4['customer_code'].isin(existing_customers)*1

C:\Users\nasta\AppData\Local\Temp\ipykernel_20996\999605109.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_4['new_customer'] = ~df_4['customer_code'].isin(existing_customers)*1


In [153]:
#treatment group: new customers in df_post during LP_period = 1
#control group: new customers in df_pre during LP_period = 1
# x: treatment (LP_period = 1)
# y: New customers per day 

#assumption: the new customer is new only on the day it visited the store for the fisrt time

#idea: measure loyalty of new customers by seeing how many times they visit the store after their first visit?
#do we have an objective if old customers started coming more often then before during the LP period?

In [24]:
#df.groupby
#df['date_code'].dt
df.sort_values(by=['date_code'],inplace=True)
df.set_index('date_code')

,store_code,period_code,redeemer_latest_ind,customer_code,customer_segment_level_1_code,customer_segment_level_2_code,revenue_after_discount_incl_vat,quantity_sold,store_type_new
date_code,,,,,,,,,
2020-06-29,212,1,0,946530,1.0,1.0,2188.0,5.718,Hypermarket
2020-06-29,365,1,1,1433460,1.0,1.0,2312.0,12.234,Hypermarket
2020-06-29,279,1,1,17980,3.0,6.0,2838.0,15.864,Hypermarket
2020-06-29,323,1,1,1365490,1.0,1.0,3156.0,12.594,Daily
2020-06-29,256,1,0,932800,3.0,6.0,310.0,2.000,Hypermarket
...,...,...,...,...,...,...,...,...,...
2022-04-03,297,4,1,1183790,1.0,1.0,5702.7,13.315,Online
2022-04-03,254,4,0,1382000,3.0,6.0,1179.0,5.000,Hypermarket
2022-04-03,417,4,0,1032270,2.0,2.0,3596.0,12.000,Daily


In [27]:
#apply regression discontinuity for predicting the effect of period_code = 4 on revenue_after_discount_incl_vat
re = smf.ols(formula = "new_customer ~ LP_period ", data = ).fit()
re.summary()


c:\Users\nasta\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1918: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


<class 'statsmodels.iolib.summary.Summary'>
"""
                                   OLS Regression Results                                  
===========================================================================================
Dep. Variable:     revenue_after_discount_incl_vat   R-squared:                       0.000
Model:                                         OLS   Adj. R-squared:                  0.000
Method:                              Least Squares   F-statistic:                       nan
Date:                             Wed, 29 Mar 2023   Prob (F-statistic):                nan
Time:                                     13:24:55   Log-Likelihood:            -1.5898e+07
No. Observations:                          1601247   AIC:                         3.180e+07
Df Residuals:                              1601246   BIC:                         3.180e+07
Df Model:                                        0                                         
Covariance Type:                         nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2818.6332      3.920    718.953      0.000    2810.949    2826.317
LP_period           0          0        nan        nan           0           0
==============================================================================
Omnibus:                  3435527.653   Durbin-Watson:                   1.967
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     119601071821.625
Skew:                          18.258   Prob(JB):                         0.00
Kurtosis:                    1341.389   Cond. No.                          inf
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [119]:
# Create a new DataFrame with the necessary columns
new_df = df[['date_code', 'period_code', 'customer_code', 'revenue_after_discount_incl_vat']]

# Aggregate the data by date and period
new_df = new_df.groupby(['date_code', 'period_code']).agg({'customer_code': 'count', 'revenue_after_discount_incl_vat': 'sum'})

# Rename the aggregated columns
new_df = new_df.rename(columns={'customer_code': 'num_customers', 'revenue_after_discount_incl_vat': 'total_revenue'})

# Create a new DataFrame with the necessary columns
week_df = df[['date_code', 'period_code']]

# Get the start date for each period
start_dates = week_df.groupby('period_code').agg({'date_code': 'min'})

# Create a dictionary mapping period codes to start dates
start_date_dict = start_dates.to_dict()['date_code']

# Define a function to calculate the week number for a given date and period
def get_week(date, period):
    start_date = start_date_dict[period]
    days_since_start = (date - start_date).days
    week = (days_since_start // 7) + 1
    return week

#Apply the get_week function to each row in the DataFrame
week_df['week'] = week_df.apply(lambda x: get_week(x['date_code'], x['period_code']), axis=1)

# Merge the week column back into the new_df DataFrame
new_df = pd.merge(new_df, week_df[['date_code', 'period_code', 'week']], on=['date_code', 'period_code'])

# Reset the index to create separate columns for date and period
new_df = new_df.reset_index()

new_df['treatment'] = ((new_df['period_code'] == 4)).astype(int)

# create a binary indicator column for the post-treatment period
new_df['post'] = (new_df['period_code'] > 2).astype(int)

# create an interaction term between treatment and post
new_df['treatment_post'] = new_df['treatment'] * new_df['post']


# create an interaction term between week and post
new_df['week_post'] = new_df['post'] * new_df['week']
#make a dummyfy the varaible week_post
new_df = pd.get_dummies(new_df, columns=['week'])



C:\Users\nasta\AppData\Local\Temp\ipykernel_32344\2846312335.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  week_df['week'] = week_df.apply(lambda x: get_week(x['date_code'], x['period_code']), axis=1)


In [129]:
# fit a linear regression model with the interaction term and other covariates
X = new_df[['post', 'treatment_post']]
X = sm.add_constant(X)
y = new_df['num_customers']
model = sm.OLS(y, X).fit()

# print the regression results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:          num_customers   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                 3.321e+04
Date:                Thu, 13 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:25:47   Log-Likelihood:            -2.5779e+07
No. Observations:             3252967   AIC:                         5.156e+07
Df Residuals:                 3252964   BIC:                         5.156e+07
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           5973.9432      0.521   1.15e

In [127]:

# set display options to show all rows and columns
pd.options.display.max_rows = None
pd.options.display.max_columns = None


In [128]:
#maximize the length of the output 
pd
# fit a linear regression model with the interaction term and other covariates
X = new_df[['post', 'treatment_post', 'week_1', 'week_2',
       'week_3', 'week_4', 'week_5', 'week_6', 'week_7', 'week_8', 'week_9',
       'week_10', 'week_11', 'week_12', 'week_13', 'week_14', 'week_15',
       'week_16', 'week_17', 'week_18', 'week_19', 'week_20']]
X = sm.add_constant(X)
y = new_df['num_customers']
model = sm.OLS(y, X).fit()

# print the regression results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:          num_customers   R-squared:                       0.117
Model:                            OLS   Adj. R-squared:                  0.117
Method:                 Least Squares   F-statistic:                 1.955e+04
Date:                Thu, 13 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:24:40   Log-Likelihood:            -2.5610e+07
No. Observations:             3252967   AIC:                         5.122e+07
Df Residuals:                 3252944   BIC:                         5.122e+07
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -2.708e+09   5.49e+10     -0.

In [ ]:

# fit a linear regression model with the interaction term and other covariates
X = new_df[['post', 'treatment_post', 'week_post_0',
       'week_post_1', 'week_post_2', 'week_post_3', 'week_post_4',
       'week_post_5', 'week_post_6', 'week_post_7', 'week_post_8',
       'week_post_9', 'week_post_10', 'week_post_11', 'week_post_12',
       'week_post_13', 'week_post_14', 'week_post_15', 'week_post_16',
       'week_post_17', 'week_post_18', 'week_post_19', 'week_post_20']]
X = sm.add_constant(X)
y = new_df['total_revenue']
model = sm.OLS(y, X).fit()

# print the regression results
print(model.summary())
# compute the standard deviations of the predictor variables
X_std = (X - X.mean()) / X.std()

# divide the coefficients by the standard deviations
coef_std = model.params / X_std.std()

# print the standardized coefficients
print(coef_std)


                            OLS Regression Results                            
Dep. Variable:          total_revenue   R-squared:                       0.165
Model:                            OLS   Adj. R-squared:                  0.165
Method:                 Least Squares   F-statistic:                 2.795e+04
Date:                Thu, 13 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:18:16   Log-Likelihood:            -5.4736e+07
No. Observations:             3252967   AIC:                         1.095e+08
Df Residuals:                 3252943   BIC:                         1.095e+08
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -7.133e+15   1.26e+16     -0.

In [130]:

# fit a linear regression model with the interaction term and other covariates
X = new_df[['post', 'treatment_post']]
X = sm.add_constant(X)
y = new_df['total_revenue']
model = sm.OLS(y, X).fit()

# print the regression results
print(model.summary())
# compute the standard deviations of the predictor variables
X_std = (X - X.mean()) / X.std()

# divide the coefficients by the standard deviations
coef_std = model.params / X_std.std()

# print the standardized coefficients
print(coef_std)


                            OLS Regression Results                            
Dep. Variable:          total_revenue   R-squared:                       0.056
Model:                            OLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                 9.682e+04
Date:                Thu, 13 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:26:24   Log-Likelihood:            -5.4935e+07
No. Observations:             3252967   AIC:                         1.099e+08
Df Residuals:                 3252964   BIC:                         1.099e+08
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           1.619e+07   4064.439   3984.